#### Load Data

In [4]:
from CanDataset import CanDataset
from dotenv import load_dotenv
import torch
import torch.nn as nn
import os

load_dotenv()
data_path = os.getenv('DATA_PATH')
dataset = CanDataset(data_path, log_verbosity=1)

Found ambient and attack directories.
Loading CAN metadata...
Parquet files found...
Found processed parquet files...
Loading processed parquet files...
Loading processing data into 'CanData' structure


#### ML model

In [5]:
class CANnoloAutoencoder(nn.Module):
    def __init__(self, embedding_dim, lstm_units, dense_units, dropout_rate, num_embeddings):
        super(CANnoloAutoencoder, self).__init__()

        # Encoder
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.encoder_dense = nn.Linear(embedding_dim+45, dense_units)
        self.encoder_dropout = nn.Dropout(dropout_rate)
        self.encoder_lstm = nn.LSTM(input_size=dense_units, hidden_size=lstm_units, num_layers=2, batch_first=True)

        # Decoder
        self.decoder_lstm = nn.LSTM(input_size=lstm_units, hidden_size=lstm_units, num_layers=2, batch_first=True)
        self.decoder_dense = nn.Linear(lstm_units, 45)
        self.decoder_output = nn.Sigmoid()  # To reconstruct the original packets

    def forward(self, can_ids, features):
        # Encoding
        embedded_ids = self.embedding(can_ids)
        # You might need to concatenate the embedded IDs with other features
        x = torch.cat([embedded_ids, features], dim=1)
        x = torch.tanh(self.encoder_dense(x))
        x = self.encoder_dropout(x)
        x, _ = self.encoder_lstm(x)

        # Decoding
        x, _ = self.decoder_lstm(x)
        x = self.decoder_dense(x)
        reconstructed = self.decoder_output(x)

        return reconstructed



#### Define config
This is what we feed to the CanDataset object to create a dataloader.

In [14]:
config = {
    "batch_size": 32,
    "delta_time_last_msg": {
        "specific_to_can_id": False,
        "records_back": 30
    },
    "delta_time_last_same_aid": {
        "specific_to_can_id": True,
        "records_back": 15
    },
}

use `get_dataloaders` on CanDataset object to get the data loaders

In [7]:
ambient_loader, validation_loader, attack_loader = dataset.get_dataloaders(config)

#### Example Data
From the config we defined:
    - Batch size of `32`
    - Keep track of the current Can ID.
    - want the last `30` `delta_time_last_msg`
    - want the last `15` `delta_time_last_same_aid`



In [8]:
example_data = ambient_loader.__getitem__(0) # input normally acts as index, but this does not really work as an index. More like get next item.
display(example_data)

(tensor([ 204,  253,   60,  519, 1225,  470,  420,  263,  263,  186,  485,  813,
          953,  541,  263,  241,  263,  186,   65,  263,  663, 1455,  813,   60,
          519, 1225,  470, 1049,  263,  186,  263,  813]),
 tensor([[0.0000e+00, 9.5367e-07, 9.0849e-03,  ..., 9.9996e-01, 9.9994e-01,
          1.0002e+00],
         [9.5367e-07, 9.0849e-03, 9.9890e-03,  ..., 1.0000e+00, 1.0001e+00,
          9.9888e-01],
         [9.0849e-03, 9.9890e-03, 2.0004e-02,  ..., 9.9979e-02, 1.0000e-01,
          1.0117e-01],
         ...,
         [9.5367e-07, 9.5367e-07, 4.3540e-03,  ..., 3.9964e-02, 3.9967e-02,
          4.0054e-02],
         [9.5367e-07, 4.3540e-03, 1.0190e-03,  ..., 1.9973e-02, 1.9998e-02,
          2.0013e-02],
         [4.3540e-03, 1.0190e-03, 3.0994e-06,  ..., 5.0041e-02, 5.0938e-02,
          4.7960e-02]]))

#### Example of 1 input

In [24]:
test_batch_can_ids, test_feature_vec = example_data

print(f'Represents Can ID: \n{test_batch_can_ids[0]}\n')
print(f'Represents Feature Vector: \n{test_feature_vec[0]}')

Represents Can ID: 
204

Represents Feature Vector: 
tensor([0.0000e+00, 9.5367e-07, 9.0849e-03, 9.9890e-03, 2.0004e-02, 9.7489e-04,
        1.9014e-02, 9.9111e-04, 6.4800e-03, 1.0200e-03, 9.5367e-07, 9.5367e-07,
        1.2046e-02, 1.0159e-03, 1.8436e-02, 1.0025e-02, 1.0021e-02, 9.9802e-04,
        1.0180e-03, 1.7963e-02, 1.7386e-02, 2.6519e-03, 1.0040e-03, 1.9073e-06,
        1.1921e-06, 1.0140e-03, 1.9073e-06, 9.5367e-07, 1.0171e-03, 9.5367e-07,
        1.0000e+00, 1.0000e+00, 1.0001e+00, 9.9998e-01, 1.0000e+00, 9.9993e-01,
        1.0001e+00, 1.0001e+00, 9.9992e-01, 1.0000e+00, 9.9998e-01, 1.0021e+00,
        9.9996e-01, 9.9994e-01, 1.0002e+00])


The `example_data` is a tuple containing a list of 32 (batch_size) Can ID's and the feature vectors defined in the config.

([`tensor containing Can ID's`],[`tensor containing features`])

In [11]:
unique_can_ids = dataset.get_unique_can_ids()
num_can_ids = len(unique_can_ids)
feature_vec_length = ambient_loader.features_len
print(f"Number of CAN IDs: {num_can_ids}")
print(f"Feature vector length: {feature_vec_length-1}") # minus one because the can id is the first

Number of CAN IDs: 105
Feature vector length: 45


In [15]:
# Hyperparameters
embedding_dim = num_can_ids  # embedding dimension should be equal to the number of CAN IDs
lstm_units = 128 # defined in canolo paper
dense_units = 256 # defined in canolo paper
dropout_rate = 0.2 # defined in canolo paper
num_embeddings = max(unique_can_ids) + 1 # not sure why + 1 rn but it works

# Model
model = CANnoloAutoencoder(embedding_dim, lstm_units, dense_units, dropout_rate, num_embeddings)

# Training parameters
batch_size = ambient_loader.batch_size
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()  # Binary Cross-Entropy Loss


In [16]:
import time
# time the training
start = time.time()
for i in range(100):
    ambient_loader.__getitem__(0)
end = time.time()
print(f"Time taken: {end-start}")
print(f"Time per batch: {(end-start)/100}")

Time taken: 6.636068105697632
Time per batch: 0.06636068105697632


In [29]:
# Running a forward pass with a batch of data
reconstructed_output = model(test_batch_can_ids, test_feature_vec)

mse_loss = torch.nn.MSELoss()
error = mse_loss(reconstructed_output, test_feature_vec)
print("Reconstruction Error:", error.item())

Reconstruction Error: 0.22565299272537231


#### Defining our loss function and optimizer

In [ ]:
loss_fn = torch.nn.MSELoss()  # Example loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Example optimizer

In [33]:
PSEUDO_EPOCH_SIZE = 500

def validate_model(model, validation_loader, loss_fn):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # No need to track gradients during validation
        for i, (can_ids, features) in enumerate(validation_loader):
            if i % 1000 == 0:
                break
            print(f"{i}\r")
            
            # Forward pass: compute the model output
            reconstructed = model(can_ids, features)
            # Compute the loss
            loss = loss_fn(reconstructed, features)  # Ensure correct target is used
            total_train_loss += loss.item()

    model.train()  # Revert to training mode
    avg_loss = total_loss / len(validation_loader)
    return avg_loss

def train_model(model, train_loader, validation_loader, loss_fn, optimizer, num_epochs, validation_interval):
    total_train_loss = 0
    pseudo_epoch = 1

    model.train()
    for i, (can_ids, features) in enumerate(train_loader):
        print(f"{i}", end="\r")

        # Forward pass: compute the model output
        reconstructed = model(can_ids, features)

        # Compute the loss
        loss = loss_fn(reconstructed, features)  # Ensure correct target is used
        total_train_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()  # Clear existing gradients
        loss.backward()  # Compute gradients
        optimizer.step()  # Update weights

        if i % PSEUDO_EPOCH_SIZE == 0:
            avg_train_loss = total_train_loss / len(train_loader)
            print(f"Epoch {pseudo_epoch}, Average Training Loss: {avg_train_loss}")
            if pseudo_epoch > num_epochs:
                break
            pseudo_epoch += 1

        if i % validation_interval == 0:
            # Validate the model at specified intervals
            validation_loss = validate_model(model, validation_loader, loss_fn)
            print(f"Psuedo Epoch {pseudo_epoch}, Validation Loss: {validation_loss}")

num_epochs = 2
validation_interval = 100 
            
train_model(model, ambient_loader, validation_loader, loss_fn, optimizer, num_epochs, validation_interval)


Epoch 1, Average Training Loss: 1.4127334638818533e-07
Psuedo Epoch 2, Validation Loss: 0.0
Psuedo Epoch 2, Validation Loss: 0.0
Psuedo Epoch 2, Validation Loss: 0.0
Psuedo Epoch 2, Validation Loss: 0.0
Psuedo Epoch 2, Validation Loss: 0.0
Epoch 2, Average Training Loss: 9.152403372752713e-05
Psuedo Epoch 3, Validation Loss: 0.0
Psuedo Epoch 3, Validation Loss: 0.0
Psuedo Epoch 3, Validation Loss: 0.0
Psuedo Epoch 3, Validation Loss: 0.0
Psuedo Epoch 3, Validation Loss: 0.0
Epoch 3, Average Training Loss: 0.00016105915937795432
